# ASTR 211 - HW06 Galaxy Classification - Sadie Seddon-Stettler

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import urllib
import requests

import numpy as np
import matplotlib.pyplot as plt

from codes.cosmology import d_l
from codes.constants import clight

from io import StringIO
from io import BytesIO

from PIL import Image

fontsize=12
plt.rc("savefig", dpi=200)
plt.rc('text', usetex=True)
plt.rc('font', size=fontsize)
plt.rc('xtick', direction='in') 
plt.rc('ytick', direction='in')
plt.rc('xtick.major', pad=5) 
plt.rc('xtick.minor', pad=5)
plt.rc('ytick.major', pad=5) 
plt.rc('ytick.minor', pad=5)

In [2]:
def read_meert_catalog(phot_type=None, datadir=None):
    """Loader for the Meert et al. 2015 catalog of improved photometric measurements
    for galaxies in the SDSS DR7 main galaxy catalog 
    input: phot_type - integer corresponding to the photometry model fit type from the catalog
        1=best fit, 2=deVaucouleurs, 3=Sersic, 4=DeVExp, 5=SerExp
    returns combined structured array with all the data
    """
    from astropy.io import fits


    if (phot_type < 1) or (phot_type > 5):
        raise Exception('unsupported type of Meert et al. photometry: %d, choose number between 1 and 5')

    # change datadir to the directory where UPenn catalog is located
    datameertnonpar = datadir+r'\Meert2015_v2\UPenn_PhotDec_nonParam_rband.fits'
    datameertnonparg = datadir+r'\Meert2015_v2\UPenn_PhotDec_nonParam_gband.fits'
    datameert = datadir+r'\Meert2015_v2\UPenn_PhotDec_Models_rband.fits'
    datasdss = datadir+r'\Meert2015_v2\UPenn_PhotDec_CAST.fits'
    datasdssmodels = datadir+r'\Meert2015_v2\UPenn_PhotDec_CASTmodels.fits'
    datameertg = datadir+r'\Meert2015_v2\UPenn_PhotDec_Models_gband.fits'
    datamorph = datadir+r'\Meert2015_v2\UPenn_PhotDec_H2011.fits' # morphology probabilities from Huertas-Company et al. 2011

    # mdata tables: 1=best fit, 2=deVaucouleurs, 3=Sersic, 4=DeVExp, 5=SerExp
    mdata = fits.open(datameert)[phot_type].data
    mdatag = fits.open(datameertg)[phot_type].data
    mnpdata = fits.open(datameertnonpar)[1].data
    mnpdatag = fits.open(datameertnonparg)[1].data
    sdata = fits.open(datasdss)[1].data
    phot_r = fits.open(datasdssmodels)[1].data
    morph = fits.open(datamorph)[1].data

    # eliminate galaxies with bad photometry
    fflag = mdata['finalflag']
    print("%d galaxies in Meert et al. sample initially"%(np.size(fflag)))

    def isset(flag, bit):
        """Return True if the specified bit is set in the given bit mask"""
        return (flag & (1 << bit)) != 0
        
    # use minimal quality cuts and flags recommended by Alan Meert
    igood = [(phot_r['petroMag'] > 0.) & (phot_r['petroMag'] < 100.) & (mnpdata['kcorr'] > 0) &
             (mdata['m_tot'] > 0) & (mdata['m_tot'] < 100) &
             (isset(fflag, 1) | isset(fflag, 4) | isset(fflag, 10) | isset(fflag, 14))]

    sdata = sdata[igood]; phot_r = phot_r[igood]; mdata = mdata[igood]
    mnpdata = mnpdata[igood]; mdatag = mdatag[igood]; mnpdatag = mnpdatag[igood]; morph = morph[igood]

    return sdata, mdata, mnpdata, phot_r, mdatag, mnpdatag, morph

In [ ]:
datadir = r'data/'
meert_data = read_meert_catalog(phot_type=3, datadir=datadir)
sdata, mdata, mnpdata, phot_r, mdatag, mnpdatag, morph = meert_data